# Amazon Bedrock Guardrails Enforcements - End-to-End Tutorial

## Overview

This notebook demonstrates how to implement **Amazon Bedrock Guardrails Enforcements**, a powerful feature that enables centralized safety controls across AWS accounts and organizations.

### What You'll Learn

- How to create guardrails with supported filters for enforcement
- How to implement organization-level guardrail enforcement
- How to implement account-level guardrail enforcement
- How to test and verify guardrail enforcement
- How to implement layered guardrails for defense in depth

### Key Concepts

**Organization-level enforcement**: Apply guardrails across OUs, individual accounts, or entire organizations using AWS Organizations policies

**Account-level enforcement**: Designate a guardrail version within an AWS account for all Bedrock model invocations

**Layered protection**: Combine organization, account, and application-specific guardrails with union of controls (most restrictive takes precedence)

### Prerequisites

- AWS account with appropriate IAM permissions
- For organization-level: AWS Organizations management account access
- For account-level: Account administrator access
- Python 3.8 or later
- boto3 library installed

---

## 1. Setup and Installation

First, let's install the required dependencies and initialize our AWS clients.

In [ ]:
# Install required packages
!pip install boto3 awscli --upgrade

In [ ]:
import boto3
import json
import time
from typing import Dict, List, Optional
from botocore.exceptions import ClientError

# Initialize AWS clients
REGION = 'us-west-2'  # Change to your preferred region

session = boto3.Session(profile_name='default')
bedrock_client = session.client('bedrock', region_name=REGION)
bedrock_runtime_client = session.client('bedrock-runtime', region_name=REGION)
organizations_client = session.client('organizations')
iam_client = session.client('iam')
sts_client = session.client('sts')

# Get current account information
account_id = sts_client.get_caller_identity()['Account']
print(f"✓ Initialized clients in region: {REGION}")
print(f"✓ Current AWS Account ID: {account_id}")

### Important API Information

This tutorial uses the following Bedrock Guardrails Enforcements APIs:

**For Resource-Based Policies:**
- Console only (API not yet available in boto3) - Attach via Amazon Bedrock Console > Guardrails

**For Account-Level Enforcement:**
- `put_enforced_guardrail_configuration()` - Enable account-level guardrail enforcement
- `list_enforced_guardrail_configurations()` - List all enforced guardrail configurations in an account
- `delete_enforced_guardrail_configuration()` - Remove account-level enforcement

**For Organization-Level Enforcement:**
- AWS Organizations APIs (`enable_policy_type`, `create_policy`, `attach_policy`)

**For Testing:**
- `apply_guardrail()` - Test guardrail behavior directly
- `invoke_model()` / `converse()` - Test with actual model invocations

---


---

## Part 1: Creating Guardrails for Enforcement

### Supported Guardrail Filters

When creating guardrails for enforcement, the following filters are supported:
- **Content filters**: Block harmful content (sexual, violence, hate, insults, misconduct)
- **Denied topics**: Prevent responses on specific topics
- **Word filters**: Block specific words or use managed word lists
- **Sensitive information filters**: Detect and block PII
- **Contextual grounding checks**: Verify responses are grounded in source material

⚠️ **IMPORTANT**: Do NOT include the automated reasoning policy - it is unsupported for guardrail enforcements and will cause runtime failures.

### Step 1.1: Create a Guardrail

Let's create a guardrail with multiple safety filters.

In [ ]:
def create_guardrail(name: str, description: str) -> Dict:
    """
    Create a guardrail with supported filters for enforcement.
    """
    print(f"Creating Guardrail: {name}")
    print("="*70)
    
    try:
        # Configure content filters (block harmful content)
        content_policy_config = {
            'filtersConfig': [
                {'type': 'SEXUAL', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                {'type': 'VIOLENCE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                {'type': 'HATE', 'inputStrength': 'MEDIUM', 'outputStrength': 'MEDIUM'},
                {'type': 'INSULTS', 'inputStrength': 'MEDIUM', 'outputStrength': 'MEDIUM'}
            ],
            'tierConfig': {
                'tierName': 'STANDARD'
            }
        }
        
        # Configure denied topics (example: financial advice)
        topic_policy_config = {
            'topicsConfig': [
                {
                    'name': 'Financial Advice',
                    'definition': 'Providing specific investment, trading, or financial planning advice',
                    'examples': [
                        'Should I invest in this stock?',
                        'What cryptocurrency should I buy?'
                    ],
                    'type': 'DENY'
                }
            ],
            'tierConfig': {
                'tierName': 'STANDARD'
            }
        }
        
        # Configure word filters (block profanity)
        word_policy_config = {
            'wordsConfig': [{'text': 'badword1'}, {'text': 'badword2'}],
            'managedWordListsConfig': [{'type': 'PROFANITY'}]
        }
        
        # Configure sensitive information filters (PII)
        sensitive_info_policy_config = {
            'piiEntitiesConfig': [
                {'type': 'EMAIL', 'action': 'BLOCK'},
                {'type': 'PHONE', 'action': 'BLOCK'},
                {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
            ]
        }
        
        response = bedrock_client.create_guardrail(
            name=name,
            description=description,
            contentPolicyConfig=content_policy_config,
            topicPolicyConfig=topic_policy_config,
            wordPolicyConfig=word_policy_config,
            sensitiveInformationPolicyConfig=sensitive_info_policy_config,
            blockedInputMessaging="I cannot process this request due to safety policies.",
            blockedOutputsMessaging="I cannot provide this response due to safety policies.",
            crossRegionConfig={
                'guardrailProfileIdentifier': 'us.guardrail.v1:0'
            },
        )
        
        guardrail_id = response['guardrailId']
        guardrail_arn = response['guardrailArn']
        
        print(f"✓ Guardrail created successfully!")
        print(f"  Guardrail ID: {guardrail_id}")
        print(f"  Guardrail ARN: {guardrail_arn}")
        
        return {
            'guardrailId': guardrail_id,
            'guardrailArn': guardrail_arn,
            'version': response.get('version', 'DRAFT')
        }
        
    except ClientError as e:
        print(f"✗ Error creating guardrail: {e}")
        raise

# Create the guardrail
guardrail = create_guardrail(
    name="EnforcementDemoGuardrail",
    description="Demonstration guardrail for enforcement tutorial"
)

### Step 1.2: Create a Guardrail Version

Versions are **required** for enforcement to ensure guardrails cannot be modified after deployment. Creating a version makes the guardrail immutable.

In [ ]:
def create_guardrail_version(guardrail_id: str, description: str = "") -> Dict:
    """
    Create an immutable version of the guardrail.
    """
    print(f"\nCreating Guardrail Version")
    print("="*70)
    
    try:
        response = bedrock_client.create_guardrail_version(
            guardrailIdentifier=guardrail_id,
            description=description or f"Version created at {time.strftime('%Y-%m-%d %H:%M:%S')}"
        )
        
        version = response['version']
        
        print(f"✓ Guardrail version created successfully!")
        print(f"  Version: {version}")
        
        # Wait for version to be ready
        print("  Waiting for version to be ready...")
        time.sleep(5)
        print("  ✓ Version ready!")
        
        return {'version': version, 'guardrailId': guardrail_id}
        
    except ClientError as e:
        print(f"✗ Error creating guardrail version: {e}")
        raise

# Create version
version_info = create_guardrail_version(
    guardrail_id=guardrail['guardrailId'],
    description="Production version for enforcement"
)

### Step 1.3: Attach Resource-Based Policies (⚠️ Console Only)

Resource-based policies (RBPs) are IAM policies attached directly to guardrail resources to grant cross-account access.

- **REQUIRED** for organization-level enforced guardrails
- **RECOMMENDED** for account-level enforced guardrails

#### ⚠️ Important: Cross-Region Inference (CRIS) Requires TWO RBPs

When your guardrail has Cross-Region Inference enabled (via `crossRegionConfig`), you must attach RBPs to **BOTH**:

1. **The guardrail itself** - Grants access to the guardrail resource
2. **The guardrail inference profile** - Grants access to the CRIS profile (e.g., `us.guardrail.v1:0`)

If you only attach the RBP to the guardrail but not the profile, member accounts will receive an `AccessDeniedException: The provided resource ARN is from a different account` error.

**To attach RBPs:**

*For the Guardrail:*
1. Go to Amazon Bedrock Console > Guardrails
2. Select your guardrail
3. Click **Create resource policy** under Resource-based policy
4. Paste the guardrail policy document (generated below) and save

*For the Guardrail Profile (required for CRIS):*
1. Go to Amazon Bedrock Console > Guardrails dashboard
2. Find the **System-defined guardrail profiles** section
3. Select the profile used by your guardrail (e.g., `us.guardrail.v1:0`)
4. Click **Create resource policy** under Resource-based policy
5. Paste the guardrail profile policy document (generated below) and save

The cells below generate the policy documents you need to copy into the console.

In [ ]:
import boto3
from botocore.exceptions import ClientError

def get_organization_id():
    try:
        response = organizations_client.describe_organization()
        return response['Organization']['Id']
    except ClientError as e:
        if e.response['Error']['Code'] == 'AWSOrganizationsNotInUseException':
            print("This account is not part of an AWS Organization")
        else:
            print(f"Error: {e}")
        return None

ORG_ID = get_organization_id()
if ORG_ID:
    print(f"Organization ID: {ORG_ID}")

In [ ]:
def generate_org_rbp_policy(guardrail_id: str, org_id: str) -> None:
    """
    Generate a resource-based policy for organization-wide access.
    This is REQUIRED for organization-level enforced guardrails.
    
    NOTE: Attach this policy via the AWS Console (API not yet available).
    """
    print(f"\nGenerating Resource-Based Policy (Organization)")
    print("="*70)
    
    policy = {
        "Version": "2012-10-17",
        "Statement": [{
            "Sid": "AllowOrganizationAccess",
            "Effect": "Allow",
            "Principal": "*",
            "Action": [
                "bedrock:ApplyGuardrail",
                "bedrock:GetGuardrail"
            ],
            "Resource": f"arn:aws:bedrock:{REGION}:{account_id}:guardrail/{guardrail_id}",
            "Condition": {
                "StringEquals": {
                    "aws:PrincipalOrgID": org_id
                }
            }
        }]
    }
    
    print(f"  Policy Type: Organization-wide")
    print(f"  Organization ID: {org_id}")
    print(f"  Guardrail ID: {guardrail_id}")
    print(f"\nPolicy Document (copy this to the Console):")
    print(json.dumps(policy, indent=2))
    
    print(f"\n⚠️  MANUAL STEP REQUIRED:")
    print(f"   Attach this policy via the AWS Console:")
    print(f"   1. Go to Amazon Bedrock Console > Guardrails")
    print(f"   2. Select guardrail with ID: {guardrail_id}")
    print(f"   3. Click 'Create resource policy' under Resource-based policy")
    print(f"   4. Paste and replace the policy document with the above, and save")

# Generate policy for organization-wide access
# Run this cell, then attach the policy via the AWS Console
generate_org_rbp_policy(
    guardrail_id=guardrail['guardrailId'],
    org_id=ORG_ID
)

#### Step 1.3.1: Generate Guardrail Profile RBP (Required for Cross-Region Inference)

If your guardrail uses Cross-Region Inference (CRIS), you **MUST** also attach an RBP to the guardrail inference profile. Without this, member accounts will receive `AccessDeniedException` errors when the organization-enforced guardrail is applied.

In [ ]:
# Guardrail Profile Identifier used in crossRegionConfig
GUARDRAIL_PROFILE_ID = 'us.guardrail.v1:0'

def generate_org_profile_rbp_policy(profile_id: str, org_id: str) -> None:
    """
    Generate a resource-based policy for the guardrail inference profile.
    This is REQUIRED for organization-level enforced guardrails when using Cross-Region Inference (CRIS).
    
    NOTE: Attach this policy via the AWS Console (API not yet available).
    """
    print(f"\nGenerating Resource-Based Policy for Guardrail Profile (Organization)")
    print("="*70)
    
    policy = {
        "Version": "2012-10-17",
        "Statement": [{
            "Sid": "AllowOrganizationProfileAccess",
            "Effect": "Allow",
            "Principal": "*",
            "Action": [
                "bedrock:ApplyGuardrail"
            ],
            "Resource": f"arn:aws:bedrock:{REGION}:{account_id}:guardrail-profile/{profile_id}",
            "Condition": {
                "StringEquals": {
                    "aws:PrincipalOrgID": org_id
                }
            }
        }]
    }
    
    print(f"  Policy Type: Organization-wide (Guardrail Profile)")
    print(f"  Organization ID: {org_id}")
    print(f"  Profile ID: {profile_id}")
    print(f"\nPolicy Document (copy this to the Console):")
    print(json.dumps(policy, indent=2))
    
    print(f"\n⚠️  MANUAL STEP REQUIRED:")
    print(f"   Attach this policy via the AWS Console:")
    print(f"   1. Go to Amazon Bedrock Console > Guardrails dashboard")
    print(f"   2. Find 'System-defined guardrail profiles' section")
    print(f"   3. Select profile: {profile_id}")
    print(f"   4. Click 'Create resource policy' under Resource-based policy")
    print(f"   5. Paste the policy document above and save")

# Generate policy for guardrail profile (required for CRIS)
# Run this cell, then attach the policy via the AWS Console
generate_org_profile_rbp_policy(
    profile_id=GUARDRAIL_PROFILE_ID,
    org_id=ORG_ID
)

In [ ]:
def generate_account_rbp_policy(guardrail_id: str) -> None:
    """
    Generate a resource-based policy for same-account access.
    This is RECOMMENDED for account-level enforced guardrails.
    
    NOTE: Attach this policy via the AWS Console (API not yet available).
    """
    print(f"\nGenerating Resource-Based Policy (Account)")
    print("="*70)
    
    policy = {
        "Version": "2012-10-17",
        "Statement": [{
            "Sid": "AllowAccountAccess",
            "Effect": "Allow",
            "Principal": {
                "AWS": f"arn:aws:iam::{account_id}:root"
            },
            "Action": [
                "bedrock:ApplyGuardrail",
                "bedrock:GetGuardrail"
            ],
            "Resource": f"arn:aws:bedrock:{REGION}:{account_id}:guardrail/{guardrail_id}"
        }]
    }
    
    print(f"  Policy Type: Same-account")
    print(f"  Account ID: {account_id}")
    print(f"  Guardrail ID: {guardrail_id}")
    print(f"\nPolicy Document (copy this to the Console):")
    print(json.dumps(policy, indent=2))
    
    print(f"\n⚠️  MANUAL STEP REQUIRED:")
    print(f"   Attach this policy via the AWS Console:")
    print(f"   1. Go to Amazon Bedrock Console > Guardrails")
    print(f"   2. Select guardrail: {guardrail_id}")
    print(f"   3. Click 'Add' under Resource-based policy")
    print(f"   4. Paste the policy document above and save")

# Generate policy for same-account access (for account-level enforcement)
# Uncomment to run if using account-level enforcement without organization:
# generate_account_rbp_policy(
#     guardrail_id=guardrail['guardrailId']
# )

In [ ]:
def verify_guardrail_access(guardrail_id: str, guardrail_version: int) -> bool:
    """
    Verify that the current identity can access the guardrail.
    Call this BEFORE enabling enforcement to avoid AccessDenied errors.
    """
    print(f"\nVerifying Guardrail Access")
    print("="*70)
    
    try:
        response = bedrock_runtime_client.apply_guardrail(
            guardrailIdentifier=guardrail_id,
            guardrailVersion=str(guardrail_version),
            source='INPUT',
            content=[{'text': {'text': 'Test access verification'}}]
        )
        print(f"✓ Successfully verified access to guardrail")
        print(f"  Guardrail ID: {guardrail_id}")
        print(f"  Version: {guardrail_version}")
        return True
    except ClientError as e:
        print(f"✗ Access verification failed: {e}")
        print(f"\n⚠️  Fix permissions before enabling enforcement!")
        return False

# Verify access before enabling enforcement
# This is strongly recommended to avoid AccessDenied errors
verify_guardrail_access(
    guardrail_id=guardrail['guardrailId'],
    guardrail_version=int(version_info['version'])
)

---

## Part 2: Organization-Level Enforcement

Organization-level enforcement allows you to apply guardrails across your entire AWS Organization, specific OUs, or individual member accounts from the management account.

### Architecture

```
Management Account
    ↓ Creates Guardrail
    ↓ Creates Organizations Policy
    ↓ Attaches to Organization/OU/Account
    ↓
Member Accounts
    → All Bedrock invocations automatically use guardrail
```

### Step 2.1: Enable Bedrock Policy Type

Before creating Bedrock policies in AWS Organizations, you must enable the Bedrock policy type.

In [ ]:
def enable_bedrock_policy_type() -> str:
    """
    Enable Bedrock policy type in AWS Organizations.
    Must be done from management account.
    
    Returns:
        root_id: The organization root ID (needed for attaching policies)
    """
    print(f"\nEnabling Bedrock Policy Type in AWS Organizations")
    print("="*70)
    
    # Get organization root ID
    roots = organizations_client.list_roots()
    root_id = roots['Roots'][0]['Id']
    
    try:
        # Enable Bedrock policy type
        organizations_client.enable_policy_type(
            RootId=root_id,
            PolicyType='BEDROCK_POLICY'
        )
        print(f"✓ Bedrock policy type enabled successfully!")
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'PolicyTypeAlreadyEnabledException':
            print(f"✓ Bedrock policy type already enabled")
        else:
            print(f"✗ Error enabling Bedrock policy type: {e}")
            raise
    
    print(f"  Root ID: {root_id}")
    return root_id

# Enable policy type and get root ID (requires management account)
ROOT_ID = enable_bedrock_policy_type()
print(f"\n✓ ROOT_ID saved for use in attach_organization_policy()")

### Step 2.2: Create AWS Organizations Bedrock Policy

Create a Bedrock policy that specifies which guardrail to enforce and how to handle input tags.

In [ ]:
def create_organization_policy(guardrail_arn: str, guardrail_version: int,
                               policy_name: str, input_tags: str = 'ignore') -> Dict:
    """
    Create an AWS Organizations Bedrock policy for guardrail enforcement.
    
    Args:
        input_tags: 'honor' or 'ignore' - set to 'ignore' to prevent bypass
    """
    print(f"\nCreating AWS Organizations Bedrock Policy")
    print("="*70)
    
    # Build the Bedrock policy document
    policy_content = {
        "bedrock": {
            "guardrail_inference": {
                REGION: {
                    "config_1": {
                        "identifier": {
                            "@@assign": f"{guardrail_arn}:{guardrail_version}"
                        },
                        "input_tags": {
                            "@@assign": input_tags
                        }
                    }
                }
            }
        }
    }
    
    try:
        response = organizations_client.create_policy(
            Content=json.dumps(policy_content),
            Description=f"Bedrock Guardrail enforcement policy for {policy_name}",
            Name=policy_name,
            Type='BEDROCK_POLICY'
        )
        
        policy_id = response['Policy']['PolicySummary']['Id']
        
        print(f"✓ Organizations policy created successfully!")
        print(f"  Policy ID: {policy_id}")
        print(f"  Policy Name: {policy_name}")
        print(f"  Input Tags Setting: {input_tags}")
        print(f"\nPolicy Content:")
        print(json.dumps(policy_content, indent=2))
        
        return {'policyId': policy_id, 'policyName': policy_name}
        
    except ClientError as e:
        print(f"✗ Error creating Organizations policy: {e}")
        raise

# Example: Create organization policy
# Uncomment to run:
org_policy = create_organization_policy(
    guardrail_arn=guardrail['guardrailArn'],
    guardrail_version=int(version_info['version']),
    policy_name="BedrockGuardrailEnforcementPolicy3",
    input_tags='ignore'
)

### Step 2.3: Attach Policy to Organization Target

Attach the policy to your organization root, specific OUs, or individual accounts.

In [ ]:
def attach_organization_policy(policy_id: str, target_id: str, target_type: str = 'root') -> None:
    """
    Attach the Bedrock policy to an organization target.
    
    Args:
        target_id: ID of the target (root, OU, or account)
        target_type: Type of target ('root', 'ou', or 'account')
    """
    print(f"\nAttaching Policy to Organization Target")
    print("="*70)
    
    try:
        organizations_client.attach_policy(
            PolicyId=policy_id,
            TargetId=target_id
        )
        
        print(f"✓ Policy attached successfully!")
        print(f"  Policy ID: {policy_id}")
        print(f"  Target ID: {target_id}")
        print(f"  Target Type: {target_type}")
        
    except ClientError as e:
        print(f"✗ Error attaching policy: {e}")
        raise

# Attach to organization root
# Uses ROOT_ID from enable_bedrock_policy_type() above
attach_organization_policy(
     policy_id=org_policy['policyId'],
     target_id=ROOT_ID,
     target_type='root'
)

---

## Part 3: Account-Level Enforcement

Account-level enforcement allows you to designate a guardrail that will be automatically applied to all Bedrock model invocations within a specific AWS account.

### Key Points

- Must be configured in **every region** where enforcement is needed
- Prevents applications from bypassing guardrails
- Can be combined with organization-level enforcement for layered protection

### Step 3.1: Enable Account-Level Enforcement

Use the `PutEnforcedGuardrailConfiguration` API to enable account-level enforcement.

In [ ]:
def put_enforced_guardrail_configuration(guardrail_id: str, guardrail_version: int,
                                        input_tags: str = "IGNORE") -> Dict:
    """
    Enable account-level guardrail enforcement.
    
    Args:
        guardrail_id: ID of the guardrail to enforce
        guardrail_version: Version number of the guardrail
        input_tags: "HONOR" or "IGNORE" - set to "IGNORE" to prevent bypass
    """
    print(f"\nEnabling Account-Level Guardrail Enforcement")
    print("="*70)
    
    try:
        response = bedrock_client.put_enforced_guardrail_configuration(
            guardrailInferenceConfig={
                'guardrailIdentifier': guardrail_id,
                'guardrailVersion': str(guardrail_version),
                'inputTags': input_tags
            }
        )
        
        print(f"✓ Account-level enforcement configured!")
        print(f"  Guardrail ID: {guardrail_id}")
        print(f"  Guardrail Version: {guardrail_version}")
        print(f"  Input Tags Setting: {input_tags}")
        print(f"  Region: {REGION}")
        
        print(f"\n⚠️  IMPORTANT: This configuration only applies to {REGION}.")
        print(f"   Repeat this step in every region where enforcement is needed.")
        
        return {
            'guardrailId': guardrail_id,
            'guardrailVersion': guardrail_version,
            'region': REGION
        }
        
    except ClientError as e:
        print(f"✗ Error configuring account-level enforcement: {e}")
        raise

# Enable account-level enforcement
# Uncomment to run:
#account_config = put_enforced_guardrail_configuration(
#     guardrail_id=guardrail['guardrailId'],
#     guardrail_version=int(version_info['version']),
#     input_tags='IGNORE'
#)


### Step 3.2: List Enforced Guardrail Configurations

View all enforced guardrail configurations in the current account and region.

In [ ]:
def list_enforced_guardrails_configuration() -> Dict:
    """
    List all enforced guardrail configurations in the current account and region.
    """
    print(f"\nListing Enforced Guardrail Configurations")
    print("="*70)
    
    try:
        response = bedrock_client.list_enforced_guardrails_configuration()
        
        print(f"✓ Retrieved enforced guardrail configurations")
        print(f"\nConfigurations in {REGION}:")
        
        if 'guardrailsConfig' in response and len(response['guardrailsConfig']) > 0:
            for config in response['guardrailsConfig']:
                print(f"\n  Config ID: {config.get('configId')}")
                print(f"  Guardrail ID: {config.get('guardrailId')}")
                print(f"  Guardrail Version: {config.get('guardrailVersion')}")
                print(f"  Input Tags: {config.get('inputTags')}")
                print(f"  Owner: {config.get('owner')}")
        else:
            print("  No enforced guardrail configurations found")
        
        return response
        
    except ClientError as e:
        # Handle the case where no configurations exist
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print(f"✓ No enforced guardrail configurations found in {REGION}")
            return {'guardrailsConfig': []}
        else:
            print(f"✗ Error listing enforced guardrail configurations: {e}")
            raise

# List configurations
# Uncomment to run:
list_enforced_guardrails_configuration()


---

## Part 4: Testing and Verification

Now let's test our guardrail enforcement to ensure it's working correctly.

### Step 4.1: Test with ApplyGuardrail API

The `ApplyGuardrail` API allows you to test guardrail behavior without invoking a model.

In [ ]:
def test_guardrail_with_apply_guardrail(guardrail_id: str, guardrail_version: int,
                                       test_content: str) -> Dict:
    """
    Test guardrail enforcement using the ApplyGuardrail API.
    """
    print(f"\nTesting Guardrail with ApplyGuardrail API")
    print("="*70)
    
    try:
        response = bedrock_runtime_client.apply_guardrail(
            guardrailIdentifier=guardrail_id,
            guardrailVersion=str(guardrail_version),
            source='INPUT',
            content=[{
                'text': {
                    'text': test_content
                }
            }]
        )
        
        print(f"✓ Guardrail assessment completed")
        print(f"\nTest Content: {test_content}")
        print(f"\nAction: {response['action']}")
        
        # Display applied guardrail details (new in enforcement feature)
        if 'assessments' in response and len(response['assessments']) > 0:
            for idx, assessment in enumerate(response['assessments']):
                if 'appliedGuardrailDetails' in assessment:
                    details = assessment['appliedGuardrailDetails']
                    print(f"\nApplied Guardrail Details (Assessment {idx + 1}):")
                    print(f"  Guardrail ID: {details.get('guardrailId')}")
                    print(f"  Guardrail Version: {details.get('guardrailVersion')}")
                    print(f"  Origin: {details.get('guardrailOrigin')}")
                    print(f"  Ownership: {details.get('guardrailOwnership')}")
        
        return response
        
    except ClientError as e:
        print(f"✗ Error testing guardrail: {e}")
        raise

# Test with safe content
test_guardrail_with_apply_guardrail(
    guardrail_id=guardrail['guardrailId'],
    guardrail_version=int(version_info['version']),
    test_content="I hate your company!"
)

### Step 4.2: Test with Bedrock Model Invocation

Test guardrail enforcement with an actual model invocation. When account-level or organization-level enforcement is configured, the guardrail will be applied automatically.

In [ ]:
def test_with_bedrock_model(model_id: str, prompt: str,
                           guardrail_id: Optional[str] = None,
                           guardrail_version: Optional[int] = None) -> Dict:
    """
    Test guardrail enforcement using the Converse API.
    
    The Converse API provides a consistent interface across all supported models
    and is the recommended approach for guardrail testing.
    """
    print(f"\nTesting with Bedrock Converse API")
    print("="*70)
    
    try:
        # Build the Converse API request parameters
        converse_params = {
            'modelId': model_id,
            'messages': [{"role": "user", "content": [{"text": prompt}]}]
        }
        
        # Add guardrail configuration if provided (request-level)
        if guardrail_id and guardrail_version:
            converse_params['guardrailConfig'] = {
                'guardrailIdentifier': guardrail_id,
                'guardrailVersion': str(guardrail_version),
                'trace': 'enabled'  # Enable trace for detailed assessment info
            }
            print(f"  Request-level guardrail: {guardrail_id}:{guardrail_version}")
        else:
            print(f"  No request-level guardrail specified")
            print(f"  Relying on account/organization enforcement")
        
        print(f"\nModel: {model_id}")
        print(f"Prompt: {prompt}")
        
        # Call the Converse API
        response = bedrock_runtime_client.converse(**converse_params)
        
        print(f"\n✓ Model invocation completed")
        
        # Display stop reason (indicates if guardrail intervened)
        stop_reason = response.get('stopReason', 'N/A')
        print(f"\nStop Reason: {stop_reason}")
        
        # Display model response
        if 'output' in response and 'message' in response['output']:
            content = response['output']['message'].get('content', [])
            if content:
                output_text = content[0].get('text', 'N/A')
                print(f"\nModel Response:")
                print(f"  {output_text[:200]}...")
        
        # Display guardrail trace if available
        if 'trace' in response and 'guardrail' in response['trace']:
            print(f"\nGuardrail Trace Available: Yes")
            if stop_reason == 'guardrail_intervened':
                print(f"  ⚠️  Guardrail blocked this request!")
        
        return response
        
    except ClientError as e:
        print(f"✗ Error invoking model: {e}")
        raise

# Test with a model (no request-level guardrail - relies on account/org enforcement)
test_with_bedrock_model(
     model_id='global.anthropic.claude-haiku-4-5-20251001-v1:0',
     prompt="I hate your company!"
)

### Step 4.3: Test Organization-Level Enforcement from Member Account

To properly verify that organization-level guardrails are being enforced, you should test from a **member account** within your AWS Organization. This proves that the guardrail configured in the management account is automatically applied to member accounts.

#### Creating the Cross-Account Test Role in the Member Account

Before running the test, create a role in the member account that the management account can assume:

**Step 1: Create the Trust Policy (in member account)**

Create a file `trust-policy.json` with the following content (replace `MANAGEMENT_ACCOUNT_ID`):

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "AWS": "arn:aws:iam::MANAGEMENT_ACCOUNT_ID:root"
      },
      "Action": "sts:AssumeRole",
      "Condition": {}
    }
  ]
}
```

**Step 2: Create the Permissions Policy (in member account)**

Create a file `bedrock-permissions.json`:

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "bedrock:InvokeModel",
        "bedrock:Converse",
        "bedrock:ApplyGuardrail"
      ],
      "Resource": "*"
    }
  ]
}
```

**Step 3: Create the Role using AWS CLI (run in member account)**

```bash
# Create the role with trust policy
aws iam create-role \\
  --role-name BedrockGuardrailTestRole \\
  --assume-role-policy-document file://trust-policy.json \\
  --description "Role for testing organization-level guardrail enforcement"

# Create and attach the permissions policy
aws iam put-role-policy \\
  --role-name BedrockGuardrailTestRole \\
  --policy-name BedrockTestPermissions \\
  --policy-document file://bedrock-permissions.json
```

**Step 4: Note the Role ARN**

The role ARN will be: `arn:aws:iam::MEMBER_ACCOUNT_ID:role/BedrockGuardrailTestRole`

#### Prerequisites Summary
- ✅ Role created in member account with trust policy allowing management account
- ✅ Role has `bedrock:InvokeModel`, `bedrock:Converse`, and `bedrock:ApplyGuardrail` permissions
- ✅ Guardrail's resource-based policy allows access from the organization (Step 1.3)
- ✅ Guardrail profile's resource-based policy allows access from the organization (Step 1.3.1) - **Required for CRIS**

In [ ]:
def test_org_guardrail_from_member_account(
    member_account_role_arn: str,
    model_id: str,
    test_prompt: str,
    session_name: str = "OrgGuardrailTest"
) -> Dict:
    """
    Test organization-level guardrail enforcement by assuming a role in a member account.
    
    This function securely assumes a cross-account role using STS AssumeRole,
    then tests that the organization-enforced guardrail is automatically applied
    to Bedrock invocations from the member account.
    
    Args:
        member_account_role_arn: ARN of the role to assume in the member account
                                 (e.g., 'arn:aws:iam::123456789012:role/BedrockTestRole')
        model_id: Bedrock model ID to use for testing
        test_prompt: Prompt to test (should trigger guardrail intervention)
        session_name: Name for the assumed role session
    
    Returns:
        Dict containing the test results and guardrail assessment details
    """
    print(f"\nTesting Organization Guardrail from Member Account")
    print("="*70)
    print(f"\nAssuming role: {member_account_role_arn}")
    
    try:
        # Step 1: Assume the role in the member account using STS
        assume_role_response = sts_client.assume_role(
            RoleArn=member_account_role_arn,
            RoleSessionName=session_name,
            DurationSeconds=900  # 15 minutes, minimum duration
        )
        
        # Extract temporary credentials
        credentials = assume_role_response['Credentials']
        assumed_account = assume_role_response['AssumedRoleUser']['Arn'].split(':')[4]
        
        print(f"✓ Successfully assumed role in account: {assumed_account}")
        print(f"  Session expires: {credentials['Expiration']}")
        
        # Step 2: Create Bedrock clients with assumed credentials
        member_bedrock_runtime = boto3.client(
            'bedrock-runtime',
            region_name=REGION,
            aws_access_key_id=credentials['AccessKeyId'],
            aws_secret_access_key=credentials['SecretAccessKey'],
            aws_session_token=credentials['SessionToken']
        )
        
        # Step 3: Test with Converse API (trace enabled to see guardrail details)
        print(f"\nTesting Bedrock invocation from member account...")
        print(f"  Model: {model_id}")
        print(f"  Prompt: {test_prompt}")
        print(f"  (No request-level guardrail - relying on organization enforcement)")
        
        response = member_bedrock_runtime.converse(
            modelId=model_id,
            messages=[{"role": "user", "content": [{"text": test_prompt}]}],
            # Note: NOT specifying guardrailConfig - organization enforcement should apply automatically
        )
        
        # Step 4: Analyze the response for guardrail intervention
        stop_reason = response.get('stopReason', 'N/A')
        print(f"\n" + "="*70)
        print("ORGANIZATION GUARDRAIL TEST RESULTS")
        print("="*70)
        print(f"\nStop Reason: {stop_reason}")
        
        # Check if guardrail intervened
        if stop_reason == 'guardrail_intervened':
            print(f"\n✓ SUCCESS: Organization-level guardrail was enforced!")
            print(f"  The guardrail from the management account automatically")
            print(f"  blocked this request in the member account.")
        else:
            print(f"\n⚠️  Guardrail did not intervene.")
            print(f"  This could mean:")
            print(f"  - The prompt did not trigger any guardrail filters")
            print(f"  - Organization policy is not attached to this account")
            print(f"  - Resource-based policy is missing or incorrect")
        
        # Display model output
        if 'output' in response and 'message' in response['output']:
            content = response['output']['message'].get('content', [])
            if content:
                output_text = content[0].get('text', 'N/A')
                print(f"\nModel Response: {output_text[:300]}")
        
        return {
            'success': True,
            'assumed_account': assumed_account,
            'stop_reason': stop_reason,
            'guardrail_intervened': stop_reason == 'guardrail_intervened',
            'response': response
        }
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        error_msg = e.response['Error']['Message']
        
        print(f"\n✗ Error: {error_code}")
        print(f"  {error_msg}")
        
        if error_code == 'AccessDenied':
            print(f"\n⚠️  Access Denied - Check the following:")
            print(f"  1. The role trust policy allows the management account to assume it")
            print(f"  2. The role has bedrock:InvokeModel and bedrock:ApplyGuardrail permissions")
            print(f"  3. The guardrail's resource-based policy grants access to the organization")
        
        return {'success': False, 'error': str(e)}


# Example: Test organization guardrail from a member account
MEMBER_ACCOUNT_ROLE_ARN = "arn:aws:iam::MEMBER_ACCOUNT_ID:role/BedrockTestRole" # Replace your member account ID role ARN
test_org_guardrail_from_member_account(
     member_account_role_arn=MEMBER_ACCOUNT_ROLE_ARN,
     model_id='global.anthropic.claude-haiku-4-5-20251001-v1:0',
     test_prompt="I hate your company!"  # Should trigger content filter
)

### Step 4.4: Test Account-Level Enforcement (Same Account)

If you configured account-level enforcement in Step 3.1, you can verify it's working by making a Bedrock invocation without specifying a guardrail. The account-enforced guardrail should automatically apply.

**Note:** This test is commented out for consistency with Step 3.1 which is also commented out. Uncomment both if testing account-level enforcement.

In [ ]:
def test_account_level_enforcement(model_id: str, test_prompt: str) -> Dict:
    """
    Test account-level guardrail enforcement.
    
    When account-level enforcement is configured (Step 3.1), the guardrail
    is automatically applied to all Bedrock invocations in this account,
    even without specifying a guardrail in the request.
    """
    print(f"\nTesting Account-Level Guardrail Enforcement")
    print("="*70)
    print(f"\nModel: {model_id}")
    print(f"Prompt: {test_prompt}")
    print(f"(No request-level guardrail - relying on account-level enforcement)")
    
    try:
        # Use Converse API without guardrailConfig
        response = bedrock_runtime_client.converse(
            modelId=model_id,
            messages=[{"role": "user", "content": [{"text": test_prompt}]}]
            # Note: NOT specifying guardrailConfig
        )
        
        stop_reason = response.get('stopReason', 'N/A')
        
        print(f"\n" + "="*70)
        print("ACCOUNT-LEVEL ENFORCEMENT TEST RESULTS")
        print("="*70)
        print(f"\nStop Reason: {stop_reason}")
        
        if stop_reason == 'guardrail_intervened':
            print(f"\n✓ SUCCESS: Account-level guardrail was enforced!")
        else:
            print(f"\n⚠️  Guardrail did not intervene.")
            print(f"  Verify account-level enforcement is configured (Step 3.1)")
        
        # Display model output
        if 'output' in response and 'message' in response['output']:
            content = response['output']['message'].get('content', [])
            if content:
                output_text = content[0].get('text', 'N/A')
                print(f"\nModel Response: {output_text[:300]}")
        
        return response
        
    except ClientError as e:
        print(f"\n✗ Error: {e}")
        raise


# Example: Test account-level enforcement
# Uncomment to run (requires Step 3.1 to be uncommented and executed first):
#
# test_account_level_enforcement(
#     model_id='global.anthropic.claude-3-haiku-20240307-v1:0',
#     test_prompt="I hate your company!"  # Should trigger content filter
# )

---

## Part 5: Layered Guardrails

One of the most powerful features of guardrail enforcements is the ability to layer multiple guardrails for defense in depth.

### How Layered Guardrails Work

When multiple guardrails apply to a single request:
1. **Organization-level enforced guardrail** (if configured)
2. **Account-level enforced guardrail** (if configured)
3. **Request-level guardrail** (specified in API call)

All applicable guardrails are enforced simultaneously. The net effect is the **UNION** of all guardrails, with the **MOST RESTRICTIVE** control taking precedence.

### Response Structure

Each guardrail assessment includes `guardrailOrigin` indicating where it came from:
- `REQUEST`: From the API call
- `ACCOUNT_ENFORCED`: From account-level configuration
- `ORGANIZATION_ENFORCED`: From organization policy

In [ ]:
def demonstrate_layered_guardrails(model_id: str, prompt: str,
                                  request_guardrail_id: str,
                                  request_guardrail_version: int) -> None:
    """
    Demonstrate layered guardrails (organization + account + request).
    Uses the Converse API with trace enabled to show detailed guardrail assessments.
    """
    print(f"\nDemonstrating Layered Guardrails")
    print("="*70)
    
    print(f"\nLayered Guardrails Concept:")
    print(f"  1. Organization-level enforced guardrail (if configured)")
    print(f"  2. Account-level enforced guardrail (if configured)")
    print(f"  3. Request-level guardrail (specified in API call)")
    print(f"\nAll applicable guardrails are enforced simultaneously.")
    print(f"The net effect is the UNION of all guardrails.")
    print(f"The MOST RESTRICTIVE control takes precedence.")
    
    try:
        # Use Converse API with guardrail trace enabled for detailed assessment info
        response = bedrock_runtime_client.converse(
            modelId=model_id,
            messages=[{"role": "user", "content": [{"text": prompt}]}],
            guardrailConfig={
                "guardrailIdentifier": request_guardrail_id,
                "guardrailVersion": str(request_guardrail_version),
                "trace": "enabled"  # Enable trace to get detailed guardrail assessments
            }
        )
        
        print(f"\n✓ Layered guardrails test completed")
        print(f"\nModel: {model_id}")
        print(f"Prompt: {prompt}")
        
        # Display stop reason
        stop_reason = response.get('stopReason', 'N/A')
        print(f"\nStop Reason: {stop_reason}")
        
        # Display model output
        if 'output' in response and 'message' in response['output']:
            content = response['output']['message'].get('content', [])
            if content:
                output_text = content[0].get('text', 'N/A')
                print(f"\nModel Response: {output_text[:200]}...")
        
        # Display guardrail trace assessments (key for layered guardrails)
        if 'trace' in response and 'guardrail' in response['trace']:
            guardrail_trace = response['trace']['guardrail']
            print(f"\n" + "="*70)
            print("GUARDRAIL INTERVENTION DETAILS")
            print("="*70)
            
            # Display action reason if guardrail intervened
            if 'actionReason' in guardrail_trace:
                print(f"\nAction Reason: {guardrail_trace['actionReason']}")
            
            # Display input assessments
            if 'inputAssessment' in guardrail_trace:
                print(f"\n--- Input Assessments ---")
                for guardrail_id, assessment in guardrail_trace['inputAssessment'].items():
                    print(f"\n  Guardrail: {guardrail_id}")
                    _display_assessment_details(assessment)
            
            # Display output assessments
            if 'outputAssessments' in guardrail_trace:
                print(f"\n--- Output Assessments ---")
                for guardrail_id, assessments in guardrail_trace['outputAssessments'].items():
                    print(f"\n  Guardrail: {guardrail_id}")
                    for assessment in assessments:
                        _display_assessment_details(assessment)
        else:
            print(f"\n⚠️  No guardrail trace found in response.")
            print(f"   This may indicate no guardrails were triggered or trace was not enabled.")
        
        print(f"\n" + "="*70)
        print(f"Check the 'guardrailOrigin' values above to see which guardrails were applied:")
        print(f"  - REQUEST: From the API call")
        print(f"  - ACCOUNT_ENFORCED: From account-level configuration")
        print(f"  - ORGANIZATION_ENFORCED: From organization policy")
        
        return response
        
    except ClientError as e:
        print(f"✗ Error in layered guardrails test: {e}")
        raise


def _display_assessment_details(assessment: Dict) -> None:
    """
    Helper function to display guardrail assessment details.
    """
    # Display applied guardrail details (origin and ownership)
    if 'appliedGuardrailDetails' in assessment:
        details = assessment['appliedGuardrailDetails']
        print(f"    Applied Guardrail Details:")
        print(f"      - ID: {details.get('guardrailId', 'N/A')}")
        print(f"      - Version: {details.get('guardrailVersion', 'N/A')}")
        print(f"      - Origin: {details.get('guardrailOrigin', 'N/A')}")
        print(f"      - Ownership: {details.get('guardrailOwnership', 'N/A')}")
    
    # Display content policy violations
    if 'contentPolicy' in assessment:
        filters = assessment['contentPolicy'].get('filters', [])
        if filters:
            print(f"    Content Policy Violations:")
            for f in filters:
                if f.get('action') == 'BLOCKED':
                    print(f"      - {f.get('type')}: {f.get('action')} (confidence: {f.get('confidence')})")
    
    # Display topic policy violations
    if 'topicPolicy' in assessment:
        topics = assessment['topicPolicy'].get('topics', [])
        if topics:
            print(f"    Topic Policy Violations:")
            for t in topics:
                if t.get('action') == 'BLOCKED':
                    print(f"      - {t.get('name')}: {t.get('action')}")
    
    # Display word policy violations
    if 'wordPolicy' in assessment:
        words = assessment['wordPolicy'].get('customWords', [])
        managed = assessment['wordPolicy'].get('managedWordLists', [])
        if words or managed:
            print(f"    Word Policy Violations:")
            for w in words:
                if w.get('action') == 'BLOCKED':
                    print(f"      - Custom word: {w.get('match')} ({w.get('action')})")
            for m in managed:
                if m.get('action') == 'BLOCKED':
                    print(f"      - Managed list ({m.get('type')}): {m.get('match')} ({m.get('action')})")
    
    # Display sensitive information policy violations
    if 'sensitiveInformationPolicy' in assessment:
        pii = assessment['sensitiveInformationPolicy'].get('piiEntities', [])
        if pii:
            print(f"    Sensitive Information Violations:")
            for p in pii:
                if p.get('action') == 'BLOCKED':
                    print(f"      - {p.get('type')}: {p.get('action')}")


# Example: Demonstrate layered guardrails
# Uncomment to run:
demonstrate_layered_guardrails(
     model_id='global.anthropic.claude-haiku-4-5-20251001-v1:0',
     prompt="I hate your company!",  # Use content that triggers guardrail
     request_guardrail_id=guardrail['guardrailId'],
     request_guardrail_version=int(version_info['version'])
)


---

## Part 6: Monitoring and Best Practices

### Monitoring Guardrail Enforcements

#### CloudWatch Metrics

Navigate to CloudWatch console and select the `AWS/Bedrock` namespace to monitor:
- **GuardrailInterventions**: Number of times guardrails blocked content
- **GuardrailEvaluations**: Total guardrail evaluations
- **GuardrailLatency**: Time taken for guardrail evaluation

#### CloudTrail Logs

Monitor these API calls in CloudTrail:
- `ApplyGuardrail`: Track guardrail usage
- `PutEnforcedGuardrailConfiguration`: Track enforcement changes
- Review `AccessDenied` errors for permission issues

#### Service Quotas

- Navigate to Service Quotas console
- Select Amazon Bedrock
- Review "Guardrails runtime" quotas
- Remember: Consumption is calculated per guardrail ARN per request

### Best Practices

1. **Always create guardrail versions** - Ensures immutability and prevents unauthorized modifications

2. **Test RBP access before enforcement** - Verify `ApplyGuardrail` API access from member accounts to avoid AccessDenied errors

3. **Set input_tags to ignore** - Prevents bypass attempts via partial content tagging

4. **Monitor Service Quotas** - Ensure sufficient runtime limits for call volume

5. **Use CloudWatch and CloudTrail** - Track guardrail usage and troubleshoot issues

6. **Plan for multi-region** - Create guardrails and configurations in all needed regions

7. **Document guardrail configurations** - Maintain clear records of which guardrails are enforced where

8. **Test with actual workloads** - Verify guardrail behavior with representative inputs before broad enforcement

### Common Pitfalls to Avoid

- ❌ Forgetting to create guardrail versions (allows modifications)
- ❌ Not setting up RBPs before organization enforcement (causes AccessDenied)
- ❌ **For CRIS**: Only attaching RBP to guardrail but not the guardrail profile (causes `AccessDeniedException: The provided resource ARN is from a different account`)
- ❌ Missing IAM permissions in member accounts
- ❌ Not configuring enforcement in all required regions
- ❌ Forgetting to enable Bedrock policy type in AWS Organizations
- ❌ Not monitoring Service Quotas for high-volume applications

### Required IAM Permissions

#### For Management Account (Organization-level)

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "bedrock:CreateGuardrail",
        "bedrock:CreateGuardrailVersion",
        "organizations:EnablePolicyType",
        "organizations:CreatePolicy",
        "organizations:AttachPolicy",
        "organizations:ListRoots"
      ],
      "Resource": "*"
    }
  ]
}
```

#### For Member Accounts

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "bedrock:ApplyGuardrail",
        "bedrock:InvokeModel",
        "bedrock:Converse"
      ],
      "Resource": "*"
    }
  ]
}
```

#### For Account-level Enforcement

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "bedrock:PutEnforcedGuardrailConfiguration",
        "bedrock:ListEnforcedGuardrailsConfiguration",
        "bedrock:DeleteEnforcedGuardrailConfiguration"
      ],
      "Resource": "*"
    }
  ]
}
```

---

## Part 7: Cleanup (Optional)

If you want to remove the resources created in this tutorial, follow these steps in order.

In [ ]:
def cleanup_account_enforcement() -> None:
    """
    Delete account-level enforced guardrail configuration.
    """
    print(f"\nDeleting Account-Level Enforcement")
    print("="*70)
    
    try:
        bedrock_client.delete_enforced_guardrail_configuration()
        
        print(f"✓ Configuration deleted successfully!")
        print(f"  Region: {REGION}")
        
    except ClientError as e:
        print(f"✗ Error deleting enforced guardrail configuration: {e}")
        raise

def cleanup_organization_policy(policy_id: str, target_id: str) -> None:
    """
    Detach and delete organization policy.
    """
    print(f"\nCleaning Up Organization Policy")
    print("="*70)
    
    try:
        # Detach policy
        organizations_client.detach_policy(
            PolicyId=policy_id,
            TargetId=target_id
        )
        print(f"✓ Policy detached from target")
        
        # Delete policy
        organizations_client.delete_policy(PolicyId=policy_id)
        print(f"✓ Policy deleted")
        
    except ClientError as e:
        print(f"✗ Error cleaning up policy: {e}")

def cleanup_guardrail(guardrail_id: str) -> None:
    """
    Delete guardrail (only after removing from enforcement).
    """
    print(f"\nDeleting Guardrail")
    print("="*70)
    
    try:
        bedrock_client.delete_guardrail(guardrailIdentifier=guardrail_id)
        print(f"✓ Guardrail deleted successfully!")
        
    except ClientError as e:
        if 'GuardrailInUse' in str(e):
            print(f"✗ Cannot delete: Guardrail is in use by enforcement configuration")
            print(f"  Remove from enforcement configuration first")
        else:
            print(f"✗ Error deleting guardrail: {e}")

# Uncomment to run cleanup:
# cleanup_account_enforcement()
# cleanup_organization_policy(policy_id=org_policy['policyId'], target_id=ROOT_ID)
# cleanup_guardrail(guardrail_id=guardrail['guardrailId'])


---

## Conclusion

Congratulations! You've completed the Amazon Bedrock Guardrails Enforcements tutorial. You've learned how to:

✅ Create guardrails with supported filters for enforcement  
✅ Implement organization-level guardrail enforcement  
✅ Implement account-level guardrail enforcement  
✅ Test and verify guardrail enforcement  
✅ Implement layered guardrails for defense in depth  
✅ Monitor and troubleshoot guardrail enforcements  

### Next Steps

1. **Implement in your organization**: Apply these patterns to your AWS accounts
2. **Customize guardrails**: Tailor filters to your specific use cases
3. **Monitor usage**: Set up CloudWatch alarms for guardrail interventions
4. **Iterate and improve**: Refine guardrails based on real-world usage

### Additional Resources

- [Amazon Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [AWS Organizations Documentation](https://docs.aws.amazon.com/organizations/)
- [Bedrock Guardrails Pricing](https://aws.amazon.com/bedrock/pricing/)

### Feedback

We'd love to hear your feedback! Please share your experience with Bedrock Guardrails Enforcements.

---

**Author**: AWS  
**License**: MIT-0  
**Last Updated**: 2025